# Crafting Recommedation System with PySpark

This Notebook uses the [movielens data set](https://grouplens.org/datasets/movielens/)

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import round, ceil, col
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml import Pipeline
import pandas as pd

In [4]:
# data = spark.read.('amazon_price.csv',inferSchema=True,header=True)
# data= spark.read.format("amazon_price").option("header", "true").load('amazon_price.xlsx
data = pd.read_json('res.json')

In [7]:
def createDataframe():

SyntaxError: invalid syntax (1065889763.py, line 1)

In [6]:
for i in range(len(data)):
    for j in range(len(data[i])):
        data1=data[i][j]
        print(data1)
        

{'url': 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A1037871XKHUZ94AZ6CB&qualifier=1669713952&id=8907680612262534&widgetName=sp_atf&url=%2FSAMSUNG-Factory-Unlocked-Android-Smartphone%2Fdp%2FB09R6FJWWS%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dsmartphone%26qid%3D1669713952%26sr%3D8-1-spons%26psc%3D1', 'asin': 'B09R6FJWWS', 'title': 'SAMSUNG Galaxy A53 5G A Series Cell Phone, Factory Unlocked Android Smartphone, 128GB, 6.5” FHD Super AMOLED Screen, Long Battery Life, US Version, Black', 'price': 349.99, 'img_url': 'https://m.media-amazon.com/images/I/71HN4P-pd5L._AC_UY218_.jpg', 'rating_stars': 4.3, 'review_count': None, 'bestseller': False, 'prime': False}
{'url': 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A1037871XKHUZ94AZ6CB&qualifier=1669713952&id=8907680612262534&widgetName=sp_atf&url=%2FSAMSUNG-Factory-Unlocked-Android-Smartphone%2Fdp%2FB09R6FJWWS%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Ds

In [37]:
userId=[]
for i in range(len(data['asin'])):
    userId.append(i+1)
data['userId']=userId

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

dataframe = pd.DataFrame(data)
dataframe['bestseller']= le.fit_transform(dataframe['bestseller']) 
print(dataframe['bestseller'].unique())

dataframe['prime']= le.fit_transform(dataframe['prime']) 
print(dataframe['prime'].unique())

[0 1]
[0]


In [7]:
data.dtypes

url              object
asin             object
title            object
price           float64
img_url          object
rating_stars    float64
review_count    float64
bestseller        int64
prime             int64
userId            int64
dtype: object

In [8]:
data1=spark.createDataFrame(data)
data1.show()

+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+
|                 url|      asin|               title|  price|             img_url|rating_stars|review_count|bestseller|prime|userId|
+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+
|https://www.amazo...|B0B3L48YH9|2022 Newest HP Pa...|  499.0|https://m.media-a...|         4.4|         NaN|         0|    0|     1|
|https://www.amazo...|B09M7LQVQ9|Lenovo IdeaPad 15...|  749.0|https://m.media-a...|         4.4|         NaN|         0|    0|     2|
|https://www.amazo...|B09J1MXS2Z|Acer Aspire 5 A51...| 464.99|https://m.media-a...|         4.5|         NaN|         0|    0|     3|
|https://www.amazo...|B08B6F1NNR|Lenovo 2022 Newes...|  388.0|https://m.media-a...|         4.4|         NaN|         0|    0|     4|
|https://www.amazo...|B08N5M7S6K|2020 Apple MacBoo...|    NaN|

In [9]:
data1.dtypes

[('url', 'string'),
 ('asin', 'string'),
 ('title', 'string'),
 ('price', 'double'),
 ('img_url', 'string'),
 ('rating_stars', 'double'),
 ('review_count', 'double'),
 ('bestseller', 'bigint'),
 ('prime', 'bigint'),
 ('userId', 'bigint')]

In [10]:
data1=data1.na.fill(value=0)
data1.show()

+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+
|                 url|      asin|               title|  price|             img_url|rating_stars|review_count|bestseller|prime|userId|
+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+
|https://www.amazo...|B0B3L48YH9|2022 Newest HP Pa...|  499.0|https://m.media-a...|         4.4|         0.0|         0|    0|     1|
|https://www.amazo...|B09M7LQVQ9|Lenovo IdeaPad 15...|  749.0|https://m.media-a...|         4.4|         0.0|         0|    0|     2|
|https://www.amazo...|B09J1MXS2Z|Acer Aspire 5 A51...| 464.99|https://m.media-a...|         4.5|         0.0|         0|    0|     3|
|https://www.amazo...|B08B6F1NNR|Lenovo 2022 Newes...|  388.0|https://m.media-a...|         4.4|         0.0|         0|    0|     4|
|https://www.amazo...|B08N5M7S6K|2020 Apple MacBoo...|    0.0|

In [11]:
data1=data1.select("*",ceil(col('price')))
# data1=data1.select("*",ceil(col('rating_stars')))


In [12]:
data1.show(10)

+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+-----------+
|                 url|      asin|               title|  price|             img_url|rating_stars|review_count|bestseller|prime|userId|CEIL(price)|
+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+-----------+
|https://www.amazo...|B0B3L48YH9|2022 Newest HP Pa...|  499.0|https://m.media-a...|         4.4|         0.0|         0|    0|     1|        499|
|https://www.amazo...|B09M7LQVQ9|Lenovo IdeaPad 15...|  749.0|https://m.media-a...|         4.4|         0.0|         0|    0|     2|        749|
|https://www.amazo...|B09J1MXS2Z|Acer Aspire 5 A51...| 464.99|https://m.media-a...|         4.5|         0.0|         0|    0|     3|        465|
|https://www.amazo...|B08B6F1NNR|Lenovo 2022 Newes...|  388.0|https://m.media-a...|         4.4|         0.0|         0|    

In [13]:
indexer=[StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['asin']]
pipeline=Pipeline(stages=indexer)
transformed=pipeline.fit(data1).transform(data1)
transformed.select(['asin','asin_index'])
transformed.show()

+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+-----------+----------+
|                 url|      asin|               title|  price|             img_url|rating_stars|review_count|bestseller|prime|userId|CEIL(price)|asin_index|
+--------------------+----------+--------------------+-------+--------------------+------------+------------+----------+-----+------+-----------+----------+
|https://www.amazo...|B0B3L48YH9|2022 Newest HP Pa...|  499.0|https://m.media-a...|         4.4|         0.0|         0|    0|     1|        499|      19.0|
|https://www.amazo...|B09M7LQVQ9|Lenovo IdeaPad 15...|  749.0|https://m.media-a...|         4.4|         0.0|         0|    0|     2|        749|       7.0|
|https://www.amazo...|B09J1MXS2Z|Acer Aspire 5 A51...| 464.99|https://m.media-a...|         4.5|         0.0|         0|    0|     3|        465|       5.0|
|https://www.amazo...|B08B6F1NNR|Lenovo 2022 Newes...|  38

In [14]:
vectorAssembler = VectorAssembler(inputCols = ['rating_stars','asin_index','bestseller','prime'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(transformed)
vhouse_df = vhouse_df.select(['features', 'CEIL(price)'])
vhouse_df.show(3)

+------------------+-----------+
|          features|CEIL(price)|
+------------------+-----------+
|[4.4,19.0,0.0,0.0]|        499|
| [4.4,7.0,0.0,0.0]|        749|
| [4.5,5.0,0.0,0.0]|        465|
+------------------+-----------+
only showing top 3 rows



In [15]:
# Smaller dataset so we will use 0.8 / 0.2
training = vhouse_df.randomSplit([0.8, 0.2])

In [16]:
training[0].show()

+------------------+-----------+
|          features|CEIL(price)|
+------------------+-----------+
|[4.4,19.0,0.0,0.0]|        499|
|     (4,[0],[4.4])|        388|
| [4.5,5.0,0.0,0.0]|        465|
| [4.4,9.0,0.0,0.0]|       1029|
|[4.5,11.0,0.0,0.0]|        500|
| [4.3,8.0,1.0,0.0]|        319|
| [4.5,4.0,0.0,0.0]|        950|
|[4.4,10.0,0.0,0.0]|        594|
| [4.8,3.0,0.0,0.0]|         26|
| [4.4,6.0,0.0,0.0]|        330|
|[4.4,12.0,0.0,0.0]|        300|
|[4.4,17.0,0.0,0.0]|        630|
|[4.4,20.0,0.0,0.0]|        499|
|[4.2,16.0,0.0,0.0]|        264|
| [4.3,2.0,0.0,0.0]|        229|
|    (4,[1],[18.0])|       1600|
|    (4,[1],[21.0])|        169|
|[3.8,14.0,0.0,0.0]|        240|
|[4.2,15.0,0.0,0.0]|        700|
+------------------+-----------+



In [17]:
# # Build the recommendation model using ALS on the training data
# als = ALS(maxIter=5, regParam=0.01, rank=25,userCol="userId", itemCol="asin_index", ratingCol="CEIL(price)")
# model = als.fit(training)

In [18]:
# # Evaluate the model by computing the RMSE on the test data
# predictions = model.transform(test)

In [19]:
# predictions.show()

In [20]:
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating_stars",predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

In [21]:
# single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [22]:
# # User had 10 ratings in the test data set 
# # Realistically this should be some sort of hold out set!
# single_user.show()

In [23]:
# reccomendations = model.transform(single_user)

In [24]:
# reccomendations.orderBy('prediction',ascending=False).show()

In [25]:
lr = LinearRegression(featuresCol = 'features', labelCol='CEIL(price)', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(training[0])
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-87.40141064725589,1.1934890443457753,-163.0091741908884,0.0]
Intercept: 849.3316466299678


In [28]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 330.758423
r2: 0.134512


In [27]:
lr_predictions = lr_model.transform(training[1])
lr_predictions.select("prediction","CEIL(price)","features").show(5)

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="CEIL(price)",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+------------------+
|        prediction|CEIL(price)|          features|
+------------------+-----------+------------------+
|473.11986309246237|        749| [4.4,7.0,0.0,0.0]|
| 497.7610794879882|        200|[4.2,13.0,0.0,0.0]|
| 430.9983645674854|          0| [4.8,1.0,0.0,0.0]|
+------------------+-----------+------------------+

R Squared (R2) on test data = -0.165327


RMSE: 330.758423
r2: 0.134512
